In [ ]:
!pip install psycopg2
import psycopg2 as ps
import pandas as pd

In [243]:
df = pd.read_csv("Andrew H Youtube.csv", index_col = 0)
df.head()

,title,published_date,views,likes,favorites,comments
video_id,,,,,,
S8nPJU9xkNw,"AMA #4: Maintain Motivation, Improve REM Sleep...",2023-02-24,73405,3346,0,156
q37ARYnRDGc,Dr. Andy Galpin: Optimal Nutrition & Supplemen...,2023-02-22,137374,4025,0,554
x4m_PdFbu-s,"How to Breathe Correctly for Optimal Health, M...",2023-02-20,210579,7639,0,1086
juD99_sPWGU,Dr. Andy Galpin: Maximize Recovery to Achieve ...,2023-02-15,169357,3853,0,407
BMTt8gSl13s,"Dr. Gina Poe: Use Sleep to Enhance Learning, M...",2023-02-13,273016,7495,0,660


In [ ]:
#connect to database
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(
                host = host_name, 
                database = dbname,
                port = port,
                user = username, 
                password = password)

    except ps.OperationalError as err:
        raise err
    else:
        print('Database Connected!')
    return conn

In [ ]:
#create new table if not exists
def create_table(curr):
    #create an object and write the SQL command in it with being commented 
    #to execute a query in the database
    create_table_command = ("""CREATE TABLE IF NOT EXISTS yt_videos(
                        id VARCHAR (255) PRIMARY KEY,
                        title TEXT NOT NULL,
                        published_date DATE NOT NULL DEFAULT CURRENT_DATE,
                        views INTEGER NOT NULL,
                        likes INTEGER NOT NULL,
                        favorites INTEGER NOT NULL,
                        comments INTEGER NOT NULL
                        )""") 
    
    curr.execute(create_table_command) #call the execute method to executing the SQL command

In [ ]:
def insert_into_table(curr, id, title, published_date, views, likes, favorites, comments):
    insert_into_videos = ("""INSERT INTO videos (id, title, published_date, views, likes, favorites, comments)
    VALUES(%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (id, title, published_date, views, likes, favorites, comments)
    curr.execute(insert_into_videos, row_to_insert)

In [ ]:
#check if video exists
def check_if_video_exists(curr,id):
    query = ("""SELECT id FROM yt_videos WHERE id = %s""")
    
    curr.execute(query, (id, ))
    return curr.fetchone() is not None

In [ ]:
#update row if video exists
def update_row(curr, id, title, views, likes, favorites, comments):
    query = ("""UPDATE yt_videos
            SET title=%s,
                views=%s,
                likes=%s,
                favorites=%s,
                comments=%s
            WHERE id=%s;""")
    vars_to_update = (id, title, views, likes, favorites, comments)
    curr.execute(query, vars_to_update)

In [ ]:
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['id'], row['title'], row['published_date'], row['views']
                          , row['likes'], row['favorites'], row['comments'])

In [241]:
def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['id', 'title', 'published_date', 'counts',
                                   'likes', 'favorites', 'comments'])
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['id']): # If video already exists then we will update
            update_row(curr,row['id'],row['title'],row['counts'],row['likes'],row['favorites'],row['comments'])
        else: # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df

In [ ]:
#database credential
host_name = "andrew-youtube-db.cootwpp9lr0f.ap-northeast-1.rds.amazonaws.com"
dbname = "db_youtube_andrew"
port = "5432"
username = "postgres"
password = "zy4dd1n4y"
conn = None

#connect to database
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

In [ ]:
#Create table
create_table(curr)

In [ ]:
conn.commit()

In [ ]:
new_vid_df = update_db(curr, df)

In [ ]:
#Insert data in db table
append_from_df_to_db(curr,new_vid_df)